![alt text](fai_gradient_logo.png)

## install dependencies

In [ ]:
#!pip install opencv-python
#!pip install ultralytics

In [ ]:
import os
import cv2
import math
import numpy as np
from ultralytics import YOLO
cap = cv2.VideoCapture('fall_2.mp4')
# Build a YOLOv9c model from pretrained weight
model = YOLO("yolov9c.pt")

In [ ]:
# Define the class names (you need to have the class names list)
classnames = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 
              'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 
              'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 
              'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 
              'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 
              'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 
              'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 
              'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 
              'scissors', 'teddy bear', 'hair drier', 'toothbrush']

In [ ]:

# Create the "run" folder if it doesn't exist
os.makedirs('run', exist_ok=True)
frame_count = 0
save_every_n_frames = 10
frame_counter = 0

# Load the warning image
warning_img = cv2.imread('warning.png', cv2.IMREAD_UNCHANGED)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    frame = cv2.resize(frame, (980, 740))
    results = model(frame)
    
    fall_detected = False
    
    for info in results:
        parameters = info.boxes
        for box in parameters:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            confidence = box.conf[0]
            class_detect = box.cls[0]
            class_detect = int(class_detect)
            class_detect_name = classnames[class_detect]
            conf = math.ceil(confidence * 100)
            height = y2 - y1
            width = x2 - x1
            threshold = height - width

            # Draw bounding box based on fall detection
            if conf > 80 and class_detect_name == 'person':
                if threshold < 0:
                    # Fall detected: draw red bounding box
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
                    cv2.putText(frame, 'Fall Detected', (x1, y1 - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
                    fall_detected = True

                    center_x = x1 + (width // 2)
                    center_y = y1 + (height // 2)

                    warning_resized = cv2.resize(warning_img, (width, height))

                    top_left_x = center_x - (warning_resized.shape[1] // 2)
                    top_left_y = center_y - (warning_resized.shape[0] // 2)

                    # Ensure the image fits within the frame boundaries
                    top_left_x = max(0, top_left_x)
                    top_left_y = max(0, top_left_y)
                    bottom_right_x = min(frame.shape[1], top_left_x + warning_resized.shape[1])
                    bottom_right_y = min(frame.shape[0], top_left_y + warning_resized.shape[0])

                    # Overlay the warning image
                    overlay = np.zeros_like(frame[top_left_y:bottom_right_y, top_left_x:bottom_right_x], dtype=np.uint8)
                    alpha_mask = warning_resized[:, :, 3] / 255.0
                    for c in range(0, 3):
                        overlay[:, :, c] = warning_resized[:, :, c] * alpha_mask + frame[top_left_y:bottom_right_y, top_left_x:bottom_right_x, c] * (1 - alpha_mask)

                    frame[top_left_y:bottom_right_y, top_left_x:bottom_right_x] = overlay

    # Save every 30th frame if a fall is detected
    # We do not want to save every frame based on the FPS being variable
    frame_counter += 1
    if fall_detected and frame_counter % save_every_n_frames == 0:
        cv2.imwrite(f'run/fall_detected_frame_{frame_count}.jpg', frame)
        frame_count += 1

    # Display the frame overly run sequentially
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('t'):
        break

cap.release()
cv2.destroyAllWindows()
